## Plotting Fractals
Now, let's try to apply what we learnt so far on some numpy computations. The exercise here uses the [Plotting Fractals tutorial form Numpy](https://numpy.org/numpy-tutorials/content/tutorial-plotting-fractals.html). For the mathmetical details, you can check out the original tutorial. Or even better, do that tutorial first before trying this one. For convinence, I have downloaded it and [included in this repo for you](./tutorial-plotting-fractals.ipynb).

Just like th original tutorial, we have to import Numpy and Matplotlib. We also have to import `jit` from Numba. If you don't ahve Matplotlib installed, uncommned that following and install it:

In [ ]:
#!pip install matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

from numba import jit
import time

## Julia set

Let's creat some fractals in the Julia set. To recap what we learnt in the NumPy tutorial, this is how we create a universial julia function that we can put in different parameters and it iterates to generate the fractal plots.

In [ ]:
def julia(mesh, c=-1, num_iter=10, radius=2):

    z = mesh.copy()
    diverge_len = np.zeros(z.shape)

    for i in range(num_iter):
        conv_mask = np.abs(z) < radius
        z[conv_mask] = np.square(z[conv_mask]) + c
        diverge_len[conv_mask] += 1

    return diverge_len

And there we created the meshes.

In [ ]:
x, y = np.meshgrid(np.linspace(-1, 1, 400), np.linspace(-1, 1, 400))
small_mesh = x + (1j * y)

x, y = np.meshgrid(np.linspace(-2, 2, 400), np.linspace(-2, 2, 400))
mesh = x + (1j * y)

Fianlly, the plot function.

In [ ]:
def plot_fractal(fractal, title='Fractal', figsize=(6, 6), cmap='rainbow', extent=[-2, 2, -2, 2]):

    plt.figure(figsize=figsize)
    ax = plt.axes()

    ax.set_title(f'${title}$')
    ax.set_xlabel('Real axis')
    ax.set_ylabel('Imaginary axis')

    im = ax.imshow(fractal, extent=extent, cmap=cmap)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax, label='Number of iterations')

And here we creat the plot, I added in a few lines to time how log it takes to make the plots.

In [ ]:
start = time.perf_counter()

output = julia(mesh, num_iter=15)
kwargs = {'title': 'f(z) = z^2 -1'}

plot_fractal(output, **kwargs)

end = time.perf_counter()

print("Elapsed (without compilation) = {}s".format((end - start)))

Now, let's try to make it faster. As we see when we created the plot, it make use of multiple interations, maybe Numba can make it faster? Let's try adding the jit no python mode there.

In [ ]:
@jit(nopython=True)
def julia_fast(mesh, c=-1, num_iter=10, radius=2):

    z = mesh.copy()
    diverge_len = np.zeros(z.shape)

    for i in range(num_iter):
        conv_mask = np.abs(z) < radius
        z[conv_mask] = np.square(z[conv_mask]) + c
        diverge_len[conv_mask] += 1

    return diverge_len

In [ ]:
start = time.perf_counter()

output = julia_fast(mesh, num_iter=15)
kwargs = {'title': 'f(z) = z^2 -1'}

plot_fractal(output, **kwargs)

end = time.perf_counter()

print("Elapsed (with compilation) = {}s".format((end - start)))

It does not compile. As we see that Numba does not like the way we use mask. The original code is designed this way to speed up calculation by avodiing element-wise calculation which is slow compare to array calculation with NumPy.

Now we have to choose, we can either use the object mode for it or we can use elementwise operations to replace the mask.

Normally, if we do not use Numba, doing element-wise operation would be much slower, however, as we see in the previous exercises, since Numba like speeding up loop operation, it maybe able to run faster. So let's give it a try.

In [ ]:
## TODO: modify the julia_fast() and use element_op() to make it able to use nopython mode
@jit(nopython=True, nogil=True)
def element_op(z, div_len, c, radius):
    pass

@jit(nopython=True,nogil=True)
def julia_fast(mesh, c=-1, num_iter=10, radius=2):
    pass

In [ ]:
# Un-commend the line below to see how we do it.
#%load element_op.py

Now let's try the new julia_fast() to see if it is faster

In [ ]:
start = time.perf_counter()

output = julia_fast(mesh, num_iter=15)
kwargs = {'title': 'f(z) = z^2 -1'}

plot_fractal(output, **kwargs)

end = time.perf_counter()

print("Elapsed (with compilation) = {}s".format((end - start)))

We see there is not much improvement. This is becuase we lost sometime for the element-wise operation so the solution is not optimal.

But don't give up, let's give the Mandelbrot set a try as well.

**Extra Task: do the same we did for Julia set with Mandelbrot set and compare the performance**